# Advent of Code Day 4

In Day 4, the objective was to determine if a set of passphrases was valid.  In part one, validity was simply a matter of whether the phrase contained only unique tokens (within itself, not among all passphrases).  In part two, valid phrases must also contain no anagrams.

In [ ]:
from utils import read_input

### Unique Tokens

Part one requires determining if a given passphrase contains only unique tokens.  Each token is separated by a whitespace character.  I'm not sure how Pythonic it is, but one easy way to determine that is simply to compare the length of the passphrase to the size of the phrase if represented as a set since a set guarnatees uniqueness among its elements.  If the lengths are equal, then the phrase has only unique chracters in it.

In [ ]:
def has_only_unique_tokens(passphrase):    
    tokens = passphrase.split(' ')
    
    return len(set(tokens)) == len(tokens)

### Anagrams

Part two required detetermining if a given passphrase contains any tokens that are anagrams of other tokens in the passphrase.  This method relies on the are_anagrams method to actually determine if a pair of tokens are anagrams.  This function is responsible for taking the phrase, splitting it, pairing up the tokens, and checking all of them for being anagrams. 

I'm not sure the properly Pythonic solution to this problem.  Ultimately for tokens t1, t2, t3...tn, we need to check t1 against t2-n, t2 against 3-n, etc up to tn-1 against tn.  I chose to do that by producing a Cartesian product of tokens, but I used enumerate to generate the index of each token alongside itself.  Then, I kept only those pairs in the product where the "left-hand" index is less than the right-hand index.  This ensures that tokens are only paired with tokens after them (plus excluding falsely checking the token against itself). If a phrase contains two copies of the same token, though, it will still properly determine they are anagrams since the two copies have different indices and I'm not performing any kind of value equality on the token itself.  

After producing the pairs, I used map to combine the pair with whether it's an anagram or not and then returned True/False based on whether there were any anagrams present.

In [ ]:
def has_anagrams(passphrase):
    tokens = passphrase.split(' ')
    
    token_pairs = [(x, y) for ix, x in enumerate(tokens) for iy, y in enumerate(tokens) if ix<iy]
    
    are_anagrams = map(lambda (x,y): ((x,y), is_anagram(x, y)), token_pairs)
    
    return len(filter(lambda (_, a): a, are_anagrams)) > 0    
    
   

### Is Anagram

This function is where the magic happens with respect to determining if something is an anagram.  The first idea that came into my head for part two was to first generate the full set of anagrams for any token and then check all other tokens in the phrase against that generated list.  Logically that would work except two potential problems.  One: generating all anagrams for a token is itself a bit of a trick.  Two: generating the anagrams has a terrible algorithmic complexity. 

Instead, this method takes 2 tokens (ostensibly from the original passphrase).  If the two don't have the same length, they can never be anagrams.  If they have the same length, I iterate through the characters in the phrase and check to see if that character is in possible_anagram.  If it's not, we can immediately say that they aren't anagrams since it suggests we've encountered the element once previously in the original phrase and the possible_anagram doesn't have another copy.  If it is, we remove it from possible_anagram.  At the end of the function, if we have "removed" all elements of possible_anagram, then it is indeed an anagram.  If any elements remain, then there were some elements not represented in phrase. 

In [ ]:
def is_anagram(phrase, possible_anagram):
        if len(phrase) != len(possible_anagram):
            return False        
        
        phrase_tokens = list(phrase)
        possible_anagram_tokens = list(possible_anagram)      
        
        for token in phrase_tokens:
            if token not in possible_anagram_tokens:
                return False
            else:
                possible_anagram_tokens.remove(token)
                
        # if possible_anagram_tokens is empty, then every 
        # token in phrase_tokens has been accounted for
        return (not possible_anagram_tokens)       
        

### Phrases with Unique Tokens

Both parts of the problem require getting a list of phrases that only contain unique terms.  For part 1, that's the entire assignment.  For part 2, we can start with that list and then apply the additional anagram rule on top of it.  

This function makes heavy use of map and filter.  First:  a map from each phrase to a tuple of the phrase and whether it has unique tokens.  Then, a filter to only keep those phrases with unique tokens and lastly, a map to return just the phrase part of tuple. This is where I wish Python had a pipe operator like functional languages do as I think it would read a bit clearer than it does.  That makes me think the Pythonic way was to split this up into 3 temporary variables instead of chaining them like this.

In [ ]:
def phrases_with_unique_tokens(phrases):   
    
    return map(lambda (p, _): p, 
                filter (lambda (_, is_unique): is_unique, 
                    map(lambda p: (p, has_only_unique_tokens(p)), phrases)))   

### Solve Part One

Read in the phrases from the file, call phrases_with_unique_tokens, then print out a message as to how many there are. 

In [ ]:
def solve_part_one():
    phrases = read_input('Input/day4.txt')
    
    valid_phrases = phrases_with_unique_tokens(phrases)    
    
    print 'Phrases with only unique tokens = {}'.format(len(valid_phrases))

### Solve Part Two

Read all the phrases, keep the ones with unique tokens, then use map and filter to determine which phrases have anagrams and simply count them

In [ ]:
def solve_part_two():

    all_phrases = read_input('Input/day4.txt')
    
    only_unique_tokens = phrases_with_unique_tokens(all_phrases)        
    
    no_anagrams = map(lambda(p, a): p,
                        filter(lambda (p, a): not a,
                                map(lambda p: (p, has_anagrams(p)), only_unique_tokens)))
                                  
    
    print 'Phrases with no anagrams = {}'.format(len(no_anagrams))
    
    

In [ ]:
solve_part_one()

In [ ]:
solve_part_two()